# Movie Recommendations with Document Similarity

Recommender systems are one of the popular and most adopted applications of machine learning. They are typically used to recommend entities to users and these entites can be anything like products, movies, services and so on. 

Popular examples of recommendations include,
- Amazon suggesting products on its website
- Amazon Prime, Netflix, Hotstar recommending movies\shows
- YouTube recommending videos to watch

Typically recommender systems can be implemented in three ways:

- Simple Rule-based Recommenders: Typically based on specific global metrics and thresholds like movie popularity, global ratings etc.
- Content-based Recommenders: This is based on providing similar entities based on a specific entity of interest. Content metadata can be used here like movie descriptions, genre, cast, director and so on
- Collaborative filtering Recommenders: Here we don't need metadata but we try to predict recommendations and ratings based on past ratings of different users and specific items.

We will be building a movie recommendation system here where based on data\metadata pertaining to different movies, we try and recommend similar movies of interest!

![](https://i.imgur.com/c7Go7d3.png)

Since our focus in not really recommendation engines but NLP, we will be leveraging the text-based metadata for each movie to try and recommend similar movies based on specific movies of interest. This falls under content-based recommenders. 

# Install Dependencies

In [1]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 266kB 5.1MB/s 
     |████████████████████████████████| 327kB 8.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85390 sha256=5ae6978e1f0a65fa83417cd4732f4d8efa60c6efd8e4b819229365fef4225eba
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Load and View Data

In [2]:
import pandas as pd

df = pd.read_csv('https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2010%20-%20Project%208%20-%20Movie%20Recommendations%20with%20Document%20Similarity/tmdb_5000_movies.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [3]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 546 to 4553
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        4800 non-null   object 
 1   tagline      4800 non-null   object 
 2   overview     4800 non-null   object 
 3   popularity   4800 non-null   float64
 4   description  4800 non-null   object 
dtypes: float64(1), object(4)
memory usage: 225.0+ KB


In [5]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


# Build a Movie Recommender System

Here you will build your own movie recommender system. We will use the following pipeline:
- Text pre-processing
- Feature Engineering
- Document Similarity Computation
- Find top similar movies
- Build a movie recommendation function


## Document Similarity

Recommendations are about understanding the underlying features which make us favour one choice over the other. Similarity between items(in this case movies) is one way to understanding why we choose one movie over another. There are different ways to calculate similarity between two items. One of the most widely used measures is __cosine similarity__ which we have already used in the previous unit.

### Cosine Similarity

Cosine Similarity is used to calculate a numeric score to denote the similarity between two text documents. Mathematically, it is defined as follows:

$$ cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [6]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

## Extract TF-IDF Features

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4800, 20471)

## Compute Pairwise Document Similarity

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006071,0.008067,0.0,0.000000,0.0,0.000000,0.025531,0.008554,0.018111,0.000000,0.000000,0.000000,0.000000,0.007439,0.010454,0.000000,0.000000,0.008190,0.008365,0.010035,0.000000,0.000000,0.050976,0.006502,0.0,0.010728,0.0,0.006908,0.000000,0.167573,0.000000,0.000000,0.000000,0.000000,0.009191,0.053475,...,0.000000,0.009711,0.006508,0.000000,0.0,0.000000,0.0,0.02841,0.0,0.000000,0.000000,0.008870,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033246,0.0,0.000000,0.0000,0.000000,0.000000,0.034092,0.018758,0.000000,0.037930,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007968,0.000000,0.0,0.012501,0.0,0.014840,0.000000,0.000000,0.000000,0.000000,0.012814,0.000000,0.000000,0.024144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008101,0.000000,0.000000,0.016898,0.0,0.017789,0.0,0.008885,0.009432,0.000000,0.000000,0.014947,0.000000,0.000000,0.000000,0.022738,...,0.000000,0.000000,0.019783,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.011407,0.011409,0.000000,0.011632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.015329,0.0,0.008367,0.0,0.000000,0.0000,0.021596,0.000000,0.000000,0.000000,0.000000,0.017564,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017178,0.000000,0.0,0.000000,0.0,0.024326,0.005471,0.018038,0.000000,0.000000,0.000000,0.005099,0.004985,0.004705,0.000000,0.004843,0.000000,0.017026,0.000000,0.003672,0.003984,0.030998,0.006959,0.000000,0.0,0.006117,0.0,0.019548,0.020365,0.009213,0.028467,0.010515,0.004198,0.006311,0.015154,0.012698,...,0.020597,0.004723,0.000000,0.016673,0.0,0.000000,0.0,0.00000,0.0,0.006625,0.000000,0.000000,0.006972,0.000000,0.010574,0.0,0.008222,0.008604,0.012782,0.015353,0.006259,0.000000,0.0,0.000000,0.0,0.010555,0.0000,0.000000,0.000000,0.000000,0.000000,0.006903,0.005024,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,0.000000,0.000000,0.000000,0.000000,0.027958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012232,0.017893,0.043639,0.000000,0.000000,0.000000,0.0,0.023127,0.0,0.000000,0.000000,0.000000,0.000000,0.009324,0.022995,0.000000,0.000000,0.044476,...,0.000000,0.000000,0.014029,0.030562,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.056761,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0000,0.018486,0.000000,0.000000,0.000000,0.060846,0.025039,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006071,0.007968,0.017178,0.000000,1.000000,0.004673,0.0,0.064581,0.0,0.000000,0.000000,0.036616,0.023883,0.014423,0.012317,0.019837,0.008171,0.004308,0.025463,0.000000,0.008909,0.023739,0.011343,0.003009,0.000000,0.026932,0.000000,0.010027,0.0,0.019560,0.0,0.028525,0.016020,0.011313,0.016914,0.022977,0.054149,0.013769,0.027401,0.005417,...,0.005089,0.015099,0.030763,0.013665,0.0,0.012626,0.0,0.00000,0.0,0.000000,0.027022,0.012027,0.005714,0.000000,0.017233,0.0,0.022330,0.007052,0.038019,0.011785,0.018614,0.021471,0.0,0.003768,0.0,0.008651,0.0095,0.000000,0.012751,0.000000,0.022064,0.019662,0.036561,0.0,0.015826,0.000000,0.076033,0.004516,0.043475,0.011465


## Get List of Movie Titles

In [9]:
movies_list = df['title'].values
movies_list, movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object), (4800,))

## Find Top Similar Movies for a Sample Movie

Let's take __Minions__ the most popular movie the the dataframe above and try and find the most similar movies which can be recommended

#### Find movie ID

In [10]:
movie_idx = np.where(movies_list == 'Minions')[0][0]
movie_idx

0

#### Get movie similarities

In [11]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00964646])

#### Get top 5 similar movie IDs

In [12]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([ 33,  60, 737, 490, 298])

#### Get top 5 similar movies

In [13]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['Despicable Me 2', 'Despicable Me',
       'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Superman',
       'Rise of the Guardians'], dtype=object)

### Build a movie recommender function to recommend top 5 similar movies for any movie 

The movie title, movie title list and document similarity matrix dataframe will be given as inputs to the function

In [14]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

# Get popular Movie Recommendations

In [15]:
popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [16]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions
Top 5 recommended Movies: ['Despicable Me 2' 'Despicable Me'
 'Teenage Mutant Ninja Turtles: Out of the Shadows' 'Superman'
 'Rise of the Guardians']

Movie: Interstellar
Top 5 recommended Movies: ['Gattaca' 'Space Pirate Captain Harlock' 'Space Cowboys'
 'Starship Troopers' 'Final Destination 2']

Movie: Deadpool
Top 5 recommended Movies: ['Silent Trigger' 'Underworld: Evolution' 'Bronson' 'Shaft' 'Don Jon']

Movie: Jurassic World
Top 5 recommended Movies: ['Jurassic Park' 'The Lost World: Jurassic Park'
 "National Lampoon's Vacation" 'The Nut Job' 'Vacation']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ["Pirates of the Caribbean: Dead Man's Chest"
 'Pirates of the Caribbean: On Stranger Tides' 'The Pirate'
 'The Pirates! In an Adventure with Scientists!' 'Joyful Noise']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle for the Planet of the Apes' 'Groove' 'The Other End of the Line'
 'Chicago Overcoat

# Movie Recommendation with Embeddings

We used count based normalized features in the previous section. Can we use word embeddings and then compute movie similarity? We definitely can! Here we will use the FastText model and train it on our corpus.

The FastText model was first introduced by Facebook in 2016 as an extension and supposedly improvement of the vanilla Word2Vec model. Based on the original paper titled ‘Enriching Word Vectors with Subword Information’ by Mikolov et al. which is an excellent read to gain an in-depth understanding of how this model works. Overall, FastText is a framework for learning word representations and also performing robust, fast and accurate text classification. The framework is open-sourced by Facebook on GitHub and claims to have the following.
- Recent state-of-the-art English word vectors.
- Word vectors for 157 languages trained on Wikipedia and Crawl.
- Models for language identification and various supervised tasks.

Though I haven’t implemented this model from scratch, based on the research paper, following is what I learnt about how the model works. In general, predictive models like the Word2Vec model typically considers each word as a distinct entity (e.g. `where`) and generates a dense embedding for the word. However this poses to be a serious limitation with languages having massive vocabularies and many rare words which may not occur a lot in different corpora. The Word2Vec model typically ignores the morphological structure of each word and considers a word as a single entity. The FastText model considers each word as a Bag of Character n-grams. This is also called as a subword model in the paper.

We add special boundary symbols < and > at the beginning and end of words. This enables us to distinguish prefixes and suffixes from other character sequences. We also include the word w itself in the set of its n-grams, to learn a representation for each word (in addition to its character n-grams). Taking the word `where` and n=3 (tri-grams) as an example, it will be represented by the character n-grams: `<wh, whe, her, ere, re>` and the special sequence `<where>` representing the whole word. Note that the sequence , corresponding to the word `<her>` is different from the tri-gram `her` from the word `where`.

Here we leverage `gensim` to build our embeddings

In [17]:
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, size=300, window=30, min_count=2, workers=4, sg=1, iter=50)

# Generate document level embeddings

Word embedding models give us an embedding for each word, how can we use it for downstream ML\DL tasks? one way is to flatten it or use sequential models. A simpler approach is to average all word embeddings for words in a document and generate a fixed-length document level emebdding

In [18]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [19]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 300)
doc_vecs_ft.shape

(4800, 300)

# Get Movie Recommendations

We will leverage cosine similarity again to generate recommendations

In [20]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.511690,0.502033,0.506710,0.529105,0.505286,0.494994,0.438024,0.404209,0.504897,0.529134,0.563268,0.510535,0.505737,0.501862,0.499107,0.504129,0.547635,0.473004,0.483251,0.488420,0.554799,0.550687,0.534033,0.556317,0.505891,0.484420,0.525187,0.526573,0.550084,0.502690,0.543700,0.502968,0.656371,0.488576,0.554642,0.582501,0.457617,0.512847,0.583850,...,0.569532,0.595099,0.489392,0.509272,0.0,0.517365,0.416551,0.540450,0.417029,0.483269,0.505652,0.566082,0.450753,0.521988,0.444165,0.460014,0.481661,0.424238,0.482620,0.489959,0.534751,0.460271,0.342498,0.584199,0.521945,0.485070,0.517082,0.513487,0.498188,0.540208,0.556174,0.460617,0.594255,0.477909,0.513204,0.493231,0.528800,0.479699,0.506051,0.564136
1,0.511690,1.000000,0.564584,0.546903,0.613255,0.525640,0.510404,0.593089,0.496603,0.533397,0.592216,0.579955,0.515759,0.492601,0.641931,0.470805,0.532121,0.579529,0.615012,0.503529,0.473162,0.581773,0.553427,0.526703,0.570293,0.505474,0.482568,0.577955,0.580888,0.615898,0.548384,0.575708,0.537827,0.460745,0.536390,0.598098,0.606696,0.467241,0.503364,0.609352,...,0.550507,0.577893,0.641728,0.534769,0.0,0.572751,0.494156,0.556322,0.455965,0.537715,0.651243,0.576193,0.521257,0.558786,0.508069,0.440113,0.520106,0.490005,0.456138,0.531013,0.556196,0.548560,0.281730,0.532088,0.518159,0.502806,0.545566,0.583033,0.442803,0.482700,0.569479,0.489590,0.535676,0.484107,0.566858,0.486407,0.492139,0.502538,0.516326,0.558858
2,0.502033,0.564584,1.000000,0.586155,0.601060,0.476012,0.519136,0.513588,0.510688,0.582253,0.587164,0.540679,0.506284,0.519322,0.574279,0.492569,0.555842,0.550439,0.526742,0.512884,0.471066,0.541079,0.573937,0.500747,0.572512,0.549321,0.469688,0.542331,0.527529,0.553680,0.574293,0.544093,0.551199,0.515161,0.578716,0.561522,0.551297,0.489434,0.584090,0.570754,...,0.574713,0.542781,0.535827,0.504098,0.0,0.521994,0.481594,0.560079,0.452589,0.549511,0.530929,0.587003,0.540125,0.539250,0.491344,0.434976,0.493345,0.503052,0.456146,0.476818,0.527479,0.526594,0.297306,0.527281,0.473510,0.454924,0.535339,0.550087,0.538759,0.525928,0.535719,0.523008,0.560396,0.477454,0.564909,0.529979,0.536122,0.530191,0.546589,0.529668
3,0.506710,0.546903,0.586155,1.000000,0.608463,0.511364,0.475941,0.513510,0.519038,0.549838,0.555876,0.533082,0.496873,0.515266,0.577770,0.567835,0.547652,0.573406,0.506623,0.494334,0.472162,0.562860,0.566941,0.564136,0.587733,0.540535,0.441363,0.508510,0.488749,0.607226,0.491082,0.530756,0.529187,0.500205,0.491427,0.584225,0.609244,0.478729,0.542518,0.595559,...,0.503172,0.514915,0.525381,0.574634,0.0,0.491561,0.422546,0.548469,0.407645,0.529683,0.500495,0.546397,0.462649,0.523307,0.547931,0.449787,0.523025,0.500486,0.456149,0.572922,0.572468,0.502155,0.339562,0.546256,0.524947,0.482767,0.546905,0.611191,0.467962,0.509152,0.531158,0.590407,0.591120,0.492992,0.543877,0.531855,0.563142,0.485562,0.510826,0.539134
4,0.529105,0.613255,0.601060,0.608463,1.000000,0.565841,0.558492,0.673265,0.565136,0.556008,0.595883,0.632311,0.547390,0.573427,0.676859,0.570275,0.654229,0.623011,0.637556,0.582396,0.538547,0.616982,0.643422,0.624256,0.597069,0.522554,0.492220,0.629889,0.614537,0.670640,0.567116,0.631352,0.622859,0.551585,0.628122,0.617075,0.654099,0.555693,0.656496,0.627782,...,0.592152,0.568140,0.648186,0.591211,0.0,0.584227,0.531031,0.590819,0.441467,0.545086,0.628599,0.608068,0.575324,0.605762,0.598547,0.542041,0.586089,0.556022,0.587532,0.621865,0.612337,0.559002,0.327621,0.609101,0.574053,0.552214,0.638012,0.558619,0.593867,0.557564,0.630200,0.620804,0.643415,0.453111,0.620320,0.548982,0.643491,0.574196,0.628554,0.627930


In [21]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions
Top 5 recommended Movies: ['Despicable Me' 'Time Bandits'
 'Rise of the Entrepreneur: The Search for a Better Way'
 'Austin Powers: The Spy Who Shagged Me' 'Despicable Me 2']

Movie: Interstellar
Top 5 recommended Movies: ['Prometheus' 'Gattaca' 'Sea Rex 3D: Journey to a Prehistoric World'
 'The Cave' 'Starship Troopers']

Movie: Deadpool
Top 5 recommended Movies: ['Fantastic Four' 'Banshee Chapter' 'Enough' 'Spider-Man 3' 'The Hunted']

Movie: Jurassic World
Top 5 recommended Movies: ['Jurassic Park' 'Jurassic Park III' 'The Lost World: Jurassic Park'
 "National Lampoon's Vacation" 'Walking With Dinosaurs']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ['Pirates of the Caribbean: On Stranger Tides'
 'The Pirates! In an Adventure with Scientists!'
 "Pirates of the Caribbean: Dead Man's Chest"
 'American Ninja 2: The Confrontation' 'The Pirate']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle for the Pla